In [ ]:
import pandas as pd

def get_soil_ph(location):
    # Load the updated Soil Type file
    file_path = "/content/drive/MyDrive/Soil Type.xlsx"
    df = pd.read_excel(file_path)

    # Ensure column names match
    df.columns = df.columns.str.strip()

    # Clean 'Region' column (remove extra spaces or hidden characters)
    df['Region'] = df['Region'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

    # Find the soil pH for the given location
    result = df[df['Region'].str.lower() == location.lower().strip()]

    if not result.empty:
        soil_type = result.iloc[0]['Soil Type']
        soil_ph = result.iloc[0]['Soil ph']
        return f"Location: {location}\nSoil Type: {soil_type}\nSoil pH: {soil_ph}"
    else:
        return "Location not found in database."

# Example usage
user_location = input("Enter location: ")
print(get_soil_ph(user_location))


Enter location: jaffna
Location: jaffna
Soil Type: Red Yellow Latasol Soil
Soil pH: 5.5


In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.1.24
    Uninstalling flatbuffers-25.1.24:
      Successfully uninstalled flatbuffers-25.1.24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from geopy.geocoders import Nominatim

def get_lat_lon(location_name):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        print("Location not found. Please enter a valid location.")
        return None, None

def fetch_weather_data(latitude, longitude):
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": "2024-01-01",
        "end_date": "2025-01-01",
        "hourly": ["temperature_2m", "relative_humidity_2m", "rain"]
    }

    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]

    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_rain = hourly.Variables(2).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )}

    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["rain"] = hourly_rain

    hourly_dataframe = pd.DataFrame(data=hourly_data)

    avg_temperature = hourly_dataframe["temperature_2m"].mean()
    avg_humidity = hourly_dataframe["relative_humidity_2m"].mean()
    avg_rain = hourly_dataframe["rain"].mean()

    print("Average Temperature (°C):", avg_temperature)
    print("Average Relative Humidity (%):", avg_humidity)
    print("Average Rainfall (mm):", avg_rain)

if __name__ == "__main__":
    location_name = input("Enter location name: ")
    lat, lon = get_lat_lon(location_name)
    if lat is not None and lon is not None:
        fetch_weather_data(lat, lon)


Enter location name: jaffna
Coordinates 9.666080474853516°N 80.03487396240234°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Average Temperature (°C): 28.278938
Average Relative Humidity (%): 78.20544
Average Rainfall (mm): 0.18700045
